# Evaluation Retrival

Um ganze sätze zu prüfen und zu vergleichen wird ebenfalls ein Cosinus-Similarity genutzt. Hierzu werden die Sätze in Vektoren umgewandelt und dann verglichen.

Die Testdaten wurden von Claude Code anhand der Produktdatei generiert. Sinnvoll wäre es, mit dem Kunden zu sprechen und echte Fragen zu sammeln.

prdukt_chunks muss so ;)

In [20]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from tqdm.notebook import tqdm
import json
import chromadb

client = chromadb.PersistentClient(path="../3-indexing/chroma_db")
collection = client.get_collection('prdukt_chunks')
model = SentenceTransformer('deepset/gbert-large')

No sentence-transformers model found with name deepset/gbert-large. Creating a new one with mean pooling.


## Queries

Alle Queries wurden mit einem LLM erstellt, die Kategorien sind von mir.

In [32]:
with open('test_queries.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)

all_queries = []
for category_name, query_list in test_data['queries'].items():
    for query_obj in query_list:
        all_queries.append(query_obj['query'])

print(f"Geladene Queries: {len(all_queries)}")

Geladene Queries: 61


## Retrieval

In [33]:
def retrieve(queries, top_k=10):
    
    queries_embedded = model.encode(queries, normalize_embeddings=True, show_progress_bar=True)
    results = []
    
    for i, query in enumerate(queries):

        result = collection.query(
            query_embeddings=[queries_embedded[i].tolist()], 
            n_results=top_k
        )

        for rank, (doc, dist, meta) in enumerate(
            zip(
                result['documents'][0], 
                result['distances'][0],
                result['metadatas'][0]
            ), 
            start=1
        ):
            results.append({
                'query': query,
                'rank': rank,
                'document': doc,
                'distance': dist,
                'chunk_type': meta.get('chunk_type'),
                'spec_categorie': meta.get('spec_category'),
                'product_id': meta.get('product_id'),
                'product_manufacturer': meta.get('product_manufacturer')
            })

    return pd.DataFrame(results)

In [ ]:
df = retrieve(all_queries)

# print(df)

Batches: 100%|██████████| 2/2 [00:06<00:00,  3.09s/it]


                                                 query  rank  \
0    Wie hoch ist der Energieverbrauch des HMFvh 40...     1   
1    Wie hoch ist der Energieverbrauch des HMFvh 40...     2   
2    Wie hoch ist der Energieverbrauch des HMFvh 40...     3   
3    Wie hoch ist der Energieverbrauch des HMFvh 40...     4   
4    Wie hoch ist der Energieverbrauch des HMFvh 40...     5   
..                                                 ...   ...   
605                   Ultra-efficient Energieeffizienz     6   
606                   Ultra-efficient Energieeffizienz     7   
607                   Ultra-efficient Energieeffizienz     8   
608                   Ultra-efficient Energieeffizienz     9   
609                   Ultra-efficient Energieeffizienz    10   

                                              document  distance   chunk_type  \
0    Mit einem jährlichen Energieverbrauch von nur ...  0.085498  description   
1    Mit einem Energieverbrauch von 0,98 kWh pro Ta...  0.092091  des

## Metriken

### Distance Metrics

Zeigt wie gut die Retivals sind.

L2-Distanz (Euklidean Distance) zwischen Query und Retrival Chunk im normalisierten Vektorspace. Je kleiner desto besser (0 = identisch, ~0,2 = sehr ähnlich, ...). 

- Ranking: Statistiken pro Rang
- Distance Gap: Abstand zwischen ersten und zehnten Rang
- Strong Results: Erbenisse mit einer Similarity > 0,3

In [35]:
distance_stats = df.groupby('rank')['distance'].agg(['mean','median','std','min','max'])
print("Ranking")
print(distance_stats)
print("\n")

rank01_mean = df[df['rank'] == 1]['distance'].mean()
rank10_mean = df[df['rank'] == 10]['distance'].mean()
gap = rank10_mean - rank01_mean
print(f"Distance Gap (Rank 1 -> 10): {gap:.4f}")

# Cousin Sim
high_confidence = (df['distance'] < 0.3).sum()
total = len(df)
print(f"Strong Results: {high_confidence}/{total} ({high_confidence/total*100:.1f}%)")


Ranking
          mean    median       std       min       max
rank                                                  
1     0.144126  0.142331  0.050768  0.064345  0.260699
2     0.149078  0.143932  0.051139  0.064511  0.260901
3     0.151926  0.145612  0.051766  0.064543  0.261605
4     0.154018  0.145864  0.052049  0.069653  0.262764
5     0.155780  0.146535  0.053114  0.069905  0.279184
6     0.157355  0.147976  0.053973  0.070319  0.294403
7     0.158463  0.148960  0.054565  0.070865  0.303758
8     0.159325  0.149165  0.054821  0.071318  0.306031
9     0.160068  0.149983  0.054991  0.071343  0.306102
10    0.160858  0.151353  0.055217  0.071368  0.309042


Distance Gap (Rank 1 -> 10): 0.0167
Strong Results: 606/610 (99.3%)


### Consistency & Stability

- Tied Sum: Anzahl der Results mit gleicher Distanz -> Führtdazu, dass das Model nicht gut unterscheiden kann.

In [36]:
duplicate_distances = df.groupby(['query', 'distance']).size()
ties = (duplicate_distances > 1).sum()
print(f"Tied Sum: {ties}")

Tied Sum: 0


### Coverage

Anzahl der unterschiedlichen Chunks über alle Queries -> Zeigt die Abdeckung der Daten in der DB - oder ob immer die selben Chunks gefunden werden.

- Unique Chunks: Anzahl der Chunks sie einmalig gefunden wurden
- Most Wanted: Eigentlich die am häufigsten gefundenen Chunks

In [37]:
unique_chunks = df['document'].nunique()
total_chunks_in_db = collection.count()
coverage = unique_chunks / total_chunks_in_db * 100
print(f"Unique Chunks: {unique_chunks}/{total_chunks_in_db} ({coverage:.1f}%)")
print("\n")

chunk_frequency = df['document'].value_counts()
most_common = chunk_frequency.head(5)
print("Most Wanted")
for doc, count in most_common.items():
    print(f"{count}x: {doc[:100]}...")

# Additional checks because specs seems prefered


Unique Chunks: 339/1800 (18.8%)


Most Wanted
11x: Sonstiges - Netzkabel: 300, Zubehör: Edelstahlregale TT90-RC für bis zu 72 Standard-Cryoboxen, Optio...
9x: Sonstiges - Netzkabel: 300, Zubehör: Edelstahlregale TT90-RC, Wifi-Bridge, Optionen: Rollen, Umluftv...
9x: Sonstiges - Netzkabel: 300, Zubehör: Edelstahlregale TT90-RC, Optionen: vier Rollen fest montiert, z...
8x: Sonstiges - Netzkabel: 300, Zubehör: Edelstahlregale TT90-RC für bis zu 72 Standard-Cryoboxen, Wifi-...
8x: Sonstiges - flexible Ausstattung: True...
